In [ ]:
#import matplotlib_hangul
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('data/train.csv', encoding = 'cp949')
test = pd.read_csv('data/test.csv', encoding = 'cp949')

submission = pd.read_csv('data/sample_submission.csv', encoding = 'cp949')

In [ ]:
display(train.shape, test.shape)

## __Data Cleansing__

In [ ]:
print('train data 결측치 비율 \n')

for col in train.columns:
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * train[col].isnull().sum() / train[col].shape[0])
    print(msg)

- train 데이터에 occpy_type에만 30.88% 결측치가 존재함.

In [ ]:
print('test data 결측치 비율 \n')

for col in test.columns:
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * test[col].isnull().sum() / test[col].shape[0])
    print(msg)

- test 데이터에 occpy_type에만 31.52% 결측치가 존재함.

In [ ]:
train['occyp_type'].value_counts(dropna=False)

### 결측치 처리

In [ ]:
train.fillna('NaN', inplace=True)
test.fillna('NaN', inplace = True)

In [ ]:
train['occyp_type'].value_counts(dropna=False)

## __EDA__

In [ ]:
train.info()

### 이산형 변수 EDA

- gender, car, reality, income_type, edu_type, family_type, FLAG_MOBIL, work_phone, phone, email,house_type,occyp_type 12개 features 

- y_train: credit 1개 feature

In [ ]:
plt.subplots(figsize = (8,8))
plt.pie(train['credit'].value_counts(), labels = train['credit'].value_counts().index, 
        autopct="%.2f%%", shadow = True, startangle = 90)
plt.title('신용 등급 비율', size=20)
plt.show()

- 0: 신용도가 좋음 
- 2: 신용도가 낮음

- 2가 많은 것으로 보아 신용도가 낮은 사람들이 많다는 것을 알 수 있음.

In [ ]:
# 등급에 따른 차이를 보기 위한 데이터 분류
train_0 = train[train['credit']==0.0]
train_1 = train[train['credit']==1.0]
train_2 = train[train['credit']==2.0]

- 신용 등급에 따른 차이를 알아보기 위해 Categorical 그래프 함수 생성

In [ ]:
# Categorical 그래프 함수 정의
def categorical_plot(column):

  f, ax = plt.subplots(1, 3, figsize=(16, 6))


  sns.countplot(x = column,
                data = train_0,
                ax = ax[0],
                order = train_0[column].value_counts().index)
  ax[0].tick_params(labelsize=12)
  ax[0].set_title('credit = 0')
  ax[0].set_ylabel('count')
  ax[0].tick_params(rotation=50)


  sns.countplot(x = column,
                data = train_1,
                ax = ax[1],
                order = train_1[column].value_counts().index)
  ax[1].tick_params(labelsize=12)
  ax[1].set_title('credit = 1')
  ax[1].set_ylabel('count')
  ax[1].tick_params(rotation=50)

  sns.countplot(x = column,
                data = train_2,
                ax = ax[2],
                order = train_2[column].value_counts().index)
  ax[2].tick_params(labelsize=12)
  ax[2].set_title('credit = 2')
  ax[2].set_ylabel('count')
  ax[2].tick_params(rotation=50)
  plt.subplots_adjust(wspace=0.3, hspace=0.3)
  plt.show()

In [ ]:
categorical_plot("gender")

- 모든 등급에서 여성이 더 많다라는 것을 알 수 있음.

In [ ]:
categorical_plot("car")

- 모든 등급에서 차를 갖고 있지 않은 사람들이 더 많음.

In [ ]:
categorical_plot("reality")

- 모든 등급에서 부동산을 소유한 사람들이 더 많음.

In [ ]:
categorical_plot("income_type")

- 모든 등급에서 직장인들이 더 많음.

- credit 0 (신용등급이 높은 등급)에서는 Student가 존재하지 않음.

- Credit 1, 2 (신용등급이 낮은 등급)에서는 Student가 아주 조금 존재함.

In [ ]:
categorical_plot("edu_type")

- 모든 등급에서 교육 수준의 순위가 같음.

- Secondary / Secondary special : 중등 특수 교육
- Higher education : 고등 교육
- Incomplete higher : 대학에서 전체 과정을 이수하지 않았고 수료증을 받지 못했다는 것을 의미
- Lower secondary : 7th ~ 9th grades 중등교육
- Academic degree : 학사학위

- 미국의 고등학생은 졸업 이후의 진로를 대학 진학과 취업 사이에서 결정함 그리고 스스로에게 필요한 수업을 선택하여 수강함.

In [ ]:
categorical_plot("family_type")

- 모든 등급에서 결혼한 사람들이 가장 많음.

- married: 결혼 
- single/not married : 미혼
- civil_marriage : 종교 의식을 하지 않는 결혼
- separated : 이혼 (배우자와 헤어진 뒤 재혼하지 않고 혼자 살고 있는 사람)
- widow : 사별 (배우자 중 한쪽이 사망한 뒤 재혼하지 않고 혼자 살고 있는 사람)

In [ ]:
categorical_plot("house_type")

- house / apartment
- with parent
- Municipal apartment
- Rented apartment
- Office apartment
- Co-op apartment

- 모든 등급에서 순서가 같고, house / aprartment를 가진 사람들이 모든 등급에서 제일 높음.

In [ ]:
categorical_plot("FLAG_MOBIL") # 변수 제거해도 될듯

- FLAG_MOBIL은 핸드폰 소유 여부

- 모든 사람들이 휴대폰을 소유하고 있음.

In [ ]:
categorical_plot("work_phone")

- work_phone : 업무용 전화 소유 여부
    
- 모든 등급에서 업무용 전화를 소유하지 않는 사람들이 더 많음.

In [ ]:
categorical_plot("phone")

- phone: 전화 소유 여부
- 모든 등급에서 가정용 전화를 소유하지 않는 사람들이 더 많음.

In [ ]:
categorical_plot("email")

- email : 이메일 소유 여부
- 모든 등급에서 이메일을 소유하지 않는 사람들이 더 많음. 

In [ ]:
categorical_plot("occyp_type")

## 연속형 변수 EDA

- child_num, income_total, DAYS_BIRTH, DAYS_EMPLOYED, family_size, begin_month 6개 features

In [ ]:
# Numerical 그래프 함수 정의
def numerical_plot(column):
  
  fig, axes = plt.subplots(1, 3, figsize=(16, 6))


  sns.distplot(train_0[column],
                ax = axes[0])
  axes[0].tick_params(labelsize=12)
  axes[0].set_title('credit = 0')
  axes[0].set_ylabel('count')

  sns.distplot(train_1[column],
                ax = axes[1])
  axes[1].tick_params(labelsize=12)
  axes[1].set_title('credit = 1')
  axes[1].set_ylabel('count')

  sns.distplot(train_2[column],
                ax = axes[2])
  axes[2].tick_params(labelsize=12)
  axes[2].set_title('credit = 2')
  axes[2].set_ylabel('count')
  plt.subplots_adjust(wspace=0.3, hspace=0.3)

In [ ]:
numerical_plot("child_num")

In [ ]:
- 신용등급이 높은 2에서 자녀의 수가 0~2개가 가장 많음.
- 신용등급에 따라 자녀수의 차이는 존재하지 않는 것으로 보임.

In [ ]:
numerical_plot("income_total")

- 신용 등급에 따라 연간 소득의 차이는 존재하지 않는 것으로 보임.

In [ ]:
numerical_plot("family_size")

- 신용도가 높은 등급에서는 왼쪽으로 치우친 그래프임.
- 모든 등급에서 보통 가족 수가 2명인 사람들이 많음.

In [ ]:
train['family_size'].value_counts()

In [ ]:
test['family_size'].value_counts()

- test셋에는 7이하까지만 있으므로 
- train 데이터셋에 7 이상인 것들은 제거해준다.

### 이상치 처리

In [ ]:
train = train[(train['family_size'] <= 7)]
train = train.reset_index(drop=True)

In [ ]:
train['family_size'].value_counts()

### X_train, y_train, X_test로 나눔

In [ ]:
X_train = train.iloc[:,:-1]
y_train = train.iloc[:, -1]

X_train = X_train.drop(columns = ['FLAG_MOBIL'])

X_test = test
test_id = X_test.index
X_test = test.drop(columns = ['FLAG_MOBIL'])

In [ ]:
X_train.shape, y_train.shape

### 중복값 처리

In [ ]:
X_train_cp = X_train.copy()
X_test_cp = X_test.copy()

In [ ]:
X = pd.concat([X_train_cp,X_test_cp])

In [ ]:
X_train.drop(columns=['begin_month','index']).shape

In [ ]:
X_train.drop(columns=['begin_month','index']).drop_duplicates().shape

In [ ]:
X = X.astype('str')

In [ ]:
X['new'] = X['child_num']+X['income_total']+X['DAYS_BIRTH']+X['DAYS_EMPLOYED']+X['family_size']+X['gender']+X['car'] + \
    X['reality']+X['income_type']+X['edu_type']+X['family_size'] + \
    X['house_type']+X['work_phone']+X['phone']+X['email']+X['occyp_type']

In [ ]:
X.drop(columns=['child_num', 'income_total' , 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'family_size','gender' , 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type',  'work_phone', 'phone', 'email', 'occyp_type'],inplace=True)

In [ ]:
X.loc[:, 'new_factorize'] = pd.factorize(X['new'])[0].reshape(-1, 1)

In [ ]:
X['new_factorize'].value_counts()

In [ ]:
X_train_cp = X.iloc[:X_train.shape[0],:]
X_test_cp = X.iloc[X_train.shape[0]:,:]

In [ ]:
X_train['id'] = X_train_cp['new_factorize']
X_test['id'] = X_test_cp['new_factorize']

In [ ]:
X_train.drop(columns='index', inplace=True)
X_test.drop(columns='index', inplace=True)

In [ ]:
X_train

In [ ]:
X_train.to_csv('data/preprocessing_train.csv', encoding='utf-8')
X_test.to_csv('data/preprocessing_test.csv', encoding='utf-8')